# Testing notebook
This is a testing notebook, serving no other purpose other than testing things out while developping.

In [8]:
import sys, os
sys.path.append("..")

import numpy as np
from random import randint

from src.models import OpenAIBot
from src.prompts.game24 import foa_step_prompt, value_prompt, value_last_step_prompt, bfs_prompt
from src.tasks.base import DATA_PATH
from src.tasks.game24 import Game24
from src.methods.agents import Agents
from src.methods.resampler import Resampler

from src.utils import empty_folder

%load_ext autoreload
%autoreload 2

bot = OpenAIBot(model="gpt-3.5-turbo-1106")
#bot = OpenAIBot(model="gpt-4")

In [ ]:
# CoT + ToT's prompt to test the model (based on ToT only 4% success rate)
from src.prompts.game24 import foa_step_prompt, value_prompt, value_last_step_prompt, bfs_prompt

input = "5 120"

prompt = foa_step_prompt.format(input=input) 
response = bot.request(prompt)

print(f"Prompt \n{prompt}")
print("----\n----\n")
print(f"Response \n{response}")

In [ ]:
# Game24 task step function

task = Game24(bot)
idx = randint(0, len(task))
task.get_input(idx)


value_numbers = []
for i in range(task.max_steps):
    task.step()
    value_number = task.evaluate()
    value_numbers.append(value_number)
summary = []

for i in range(len(task.steps)):
    temp = task.steps[i] + f" [Value : {value_numbers[i]}]"
    summary.append(temp)

print(f"Input: {task.input}")
print("\n".join(summary))

In [ ]:
# Resampler

values = np.array([20, 10, 5, 1, 0.0001])
resampler = Resampler()

# Normalized resampling
draw = resampler.resample(values, resample_method="normalization")
print(f"Normalized : {draw}")


# Greedy
draw = resampler.resample(values, resample_method="greedy")
print(f"Greedy : {draw}")


In [ ]:
# Multiple agents

idx_input = 8
n_evaluations = 2
n_agents = 5

# Create agents
agents = Agents(task=Game24, idx_input=idx_input, n_agents=n_agents, model=bot)

# Run agents
for i in range(agents.max_steps-1):
    agents.step()
    agents.evaluate(n=n_evaluations)
    agents.resample()

# Final step : Finish answers format + Choose best answer
agents.step() 
agents.evaluate()
best_agent = agents.choose()

# Log results
current_path = os.getcwd()
log_path = os.path.join(os.path.dirname(current_path), "logs")
empty_folder(log_path)
agents.create_log(repo_path=log_path)

In [ ]:
# Test results funciton

idx_input = 8
n_evaluations = 3
n_agents = 5

# Create agents
agents = Agents(task=Game24, idx_input=idx_input, n_agents=n_agents, model=bot)

# Run agents
for i in range(agents.max_steps-1):
    agents.step()
    agents.evaluate(n=n_evaluations)
    agents.resample()

# Final step : Finish answers format + Choose best answer
agents.step() 

print("\n\n".join(["\n".join(agent.steps) for agent in agents.agents]))
print(f"\nResults : {agents.test_output()}")